In [1]:
%matplotlib inline

from pathlib import Path
import sys
sys.path.append("../../")

import anamic

import numpy as np
import matplotlib.pyplot as plt
import tqdm

data_dir = Path('/home/hadim/Drive/Data/Fake Microtubules/anamic')
data_dir.mkdir(parents=True, exist_ok=True)

../../anamic/transformations.py:1916: UserWarning: No module named 'anamic._transformations'
  warnings.warn(str(err))


In [ ]:
taper_lengths = [0, 500, 1000, 1500, 2000, 2500, 3000]
n_mt_per_condition = 5

n_pf = 13
mt_length_nm = 6000  # nm

# Set parameters for the image generation.
parameters = {}
parameters['labeling_ratio'] = 0.1  # from 0 to 1

parameters['pixel_size'] = 110  # nm/pixel
parameters['x_offset'] = 1500  # nm
parameters['y_offset'] = 1500  # nm

parameters['psf_size'] = 135  # nm

parameters['signal_mean'] = 700
parameters['signal_std'] = 100
parameters['bg_mean'] = 500
parameters['bg_std'] = 24
parameters['noise_factor'] = 1

parameters['snr_line_width'] = 3  # pixel

for taper_length in tqdm.tqdm_notebook(taper_lengths, total=len(taper_lengths), leave=False):
    
    current_dir = data_dir / f'taper_length_nm_{taper_length:04d}'
    current_dir.mkdir(parents=True, exist_ok=True)
    
    # Generate the dimers
    dimers = anamic.simulator.dimers_builder(n_pf, mt_length_nm, taper_length)
    
    # Create the simulator object
    ms = anamic.simulator.MicrotubuleSimulator(dimers)
    
    # Update the parameters
    ms.parameters.update(parameters)
    
    for i in tqdm.tqdm_notebook(range(n_mt_per_condition), leave=False):
        
        basename = f'microtubule_{i:04d}'
        
        # Build a single microtubule image
        ms.build_all(apply_random_z_rotation=True, show_progress=False)
        
        # Save image, parameters and dimer positions.
        ms.save_positions(current_dir / (basename + ".csv"))
        ms.save_parameters(current_dir / (basename + ".json"))
        ms.save_image(current_dir / (basename + ".tif"))